___
# Ciência dos Dados - PROJETO 3

___
### *Bruno Kaczelnik, Guilherme Lotaif, Renato Yasuo Chopard Tajima, Thiago Verardo*
___

# Análise de Airbnb nas maiores cidades americanas

___

## 1. Introdução

   Neste Projeto será feita uma análise dos dados que possuimos de Airbnb nos Estados Unidos, e a partir dos dados que julgarmos pertinentes ao nosso estudo. Serão usados diferentes **métodos de predição** para descobrir`o preço de um aluguel dependendo das características fornecidas`, após usarmos os diferentes métodos, vamos comparar os resultados de cada método, finalmente poderemos concluir o projeto..

   Um pouco mais sobre a plataforma digital Airbnb: Ela é utilizada para efetuar o aluguel de uma casa ou apartamento de outros usuários da própria plataforma. Com anúncios em 192 países, podemos concluir que é uma plataforma é bem grande, somente nos Estados Unidos existem aproximadamente 600.000 possiveis locais para aluguel. Com tantas opções, a escolha que o usuário faz quanto ao local é feita baseada nas **informações fornecidas pelo propretario**, tais informações podem variar desde quantos banheiros a residência possui, até se ela possui acesso a uma rede Wifi.

**Fonte do dataset:** www.kaggle.com/rudymizrahi/airbnb-listings-in-major-us-cities-deloitte-ml

___
## 2. Minerando dados e características do dataset

O dataset que vamos utilizar nesse projeto possui diversas colunas com informações dos Airbnb nos Estados Unidos, portannto temos que percorrer todas essas colunas para fazer uma limpeza, e deixar somente as informações que serão pertinentes a nossa análise.

**Importando as bibliotecas que serão utilizadas:**

In [1]:
#Importando bibliotecas:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import pandas_profiling
import matplotlib.pyplot as plt
from scipy.stats import norm,probplot

plt.style.use('ggplot')

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/guilherme/Downloads/Projeto-3-CDD---DPA-master


### - Base de dados

In [3]:
#Importado o arquivo de treinamento:
df = pd.read_csv('train.csv')

In [4]:
#Análisando o tamanho do dataframe:
linhas, colunas = df.shape
print("O Dataframe possui {0} linhas por {1} colunas.".format(linhas, colunas))

O Dataframe possui 74111 linhas por 29 colunas.


...

#### Limpando o dataframe para ser análisado:
Esta etapa consiste em uma preparação do dataframe para facilitar a análise no mesmo, assim evitando ocorrências de complicações ou erros que atrapalhem futuramento os nosssos classificadores. Portanto iremos limpar os titulos de cada coluna, vamos remover os valores nulos de cada categoria, assim como colunas desnecessárias.

In [5]:
#Removendo os espaços em branco dos nomes das colunas:
df.columns = [espaços.strip() for espaços in df.columns.tolist()]

In [6]:
#Removendo colunas que não sao pertinentes ao estudo:
df1 = df.drop(["latitude","longitude","name","thumbnail_url","id","description","log_price"],axis=1);

In [7]:
#Apagando valores nulos que podem causar problemas posteriores:
df1 = df.dropna(axis=0, subset=['bathrooms','first_review','host_has_profile_pic','host_identity_verified',
                               'host_response_rate','host_since','last_review','neighbourhood','review_scores_rating',
                               'zipcode','bedrooms','beds'])

In [10]:
#Corrigindo simplificação no arquivo:
df1.loc[(df1["instant_bookable"] == "f"),"instant_bookable"] = "False"
df1.loc[(df1["instant_bookable"] == "t"),"instant_bookable"] = "True"

df1.loc[(df1["host_has_profile_pic"] == "f"),"host_has_profile_pic"] = "False"
df1.loc[(df1["host_has_profile_pic"] == "t"),"host_has_profile_pic"] = "True"


df1.loc[(df1["host_identity_verified"] == "f"),"host_identity_verified"] = "False"
df1.loc[(df1["host_identity_verified"] == "t"),"host_identity_verified"] = "True"

In [11]:
df1.sample(2)

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
54453,19334358,4.077537,Apartment,Private room,"{Internet,""Wireless Internet"",""Wheelchair acce...",2,1.0,Real Bed,flexible,False,...,33.999740,-118.436138,"Cozy Room with Private Bath, close to Venice",Mar Vista,6,97.0,https://a0.muscache.com/im/pictures/47638b66-e...,90066,1.0,1.0
14231,17206784,5.703782,Condominium,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",3,1.0,Real Bed,flexible,False,...,40.744327,-73.985456,Light filled One bedroom apartment,Flatiron District,2,100.0,https://a0.muscache.com/im/pictures/008e6862-0...,10016,1.0,1.0


### CHECAR DESCRIPTION

#### Separando os atributos na coluna de características:
<br>
Para um melhor rendimento dos métodos de prdição, vamos separar cada atributo para que eles possam ser análisados e comparados de maneira unitária, e para isso vamos juntar todas palavras em listas para cada anúncio, e depois comparar não só os itens quanto o tamanho das listas também.

In [ ]:
'''#Criando uma lista para as palavras dos emails SPAM:
dic_amenities = {}
#Criando a variável contador para o total de palavras SPAM:
contador_amenities = 0

#Criando um loop para atribuir os emails SPAM a uma variável:
for qualidades in df.amenities:
    #Criando um loop para alocar as palavras de SPAM no dicionário:
    for caracteristica in qualidades:
        #print(caracteristica)
        if caracteristica not in dic_amenities:
            print(caracteristica)
            dic_amenities[caracteristica] =1
            contador_amenities +=1
        else:
            dic_amenities[caracteristica] +=1
            contador_amenities +='''

<br>

### - Análise descritiva

Após realizarmos o filtro, deve-se realizar uma análise exploratória dos dados, com o  objetivo de achar as váriaveis que mais influenciam no nosso objetivo e que assim possam nos ajudar a prever qual será a avaliação de um hotel aleatório. Ela será feita com o auxílio do pandas_profiling e seaborn.


In [12]:
#utilizandoo o pandas_profiling
#df é o dataframe após o filtro
pandas_profiling.ProfileReport(df1)

Number of variables,30
Number of observations,42775
Total Missing (%),0.3%
Total size in memory,9.5 MiB
Average record size in memory,233.0 B
Numeric,11
Categorical,18
Boolean,1
Date,0
Text (Unique),0
Rejected,0


___
## 3. Modelos de predição

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [15]:
#Removendo colunas que não sao pertinentes ao estudo:
df2 = df.drop(["latitude","longitude","name","thumbnail_url","id","description"],axis=1);

In [16]:
#Dividindo os dados do dataframe em teste e treino:
X_train, X_test, y_train, y_test = train_test_split(df, df['log_price'], test_size = 0.33, random_state = 0)                           

#Chamando o classificador:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, 
                                   max_depth=8, min_samples_leaf=4)
#Fazendo um fit nos dados de treino:
clf_gini.fit(X_train, y_train)   

y_pred = clf_gini.predict(X_test)

#Descobrindo a acurácia do classificador:
acc = accuracy_score(y_test, y_pred)

acc = acc*100
print("A acurácia é de {:.2f}%".format(acc))

ValueError: could not convert string to float: 'Apartment'

o MODELO DO RANDOM FOREST (Para comparação)

___
## 4. Processo e estatísticas de validação

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## 5. Conclusão

___
## 6. Referências bibliográficas